This cell is responsible for managing imports.

In [1]:
from qiskit import Aer          # for simulator backend
from qiskit_machine_learning.algorithms import QSVC # quantum support vector classifier class
from qiskit_machine_learning.kernels.quantum_kernel import QuantumKernel # wraps feature map and backend combination to give to QSVC
import qiskit.circuit.library   # for feature maps
import numpy as np
import joblib                   # for persistence, caching, and parallelism

# for data sets and data set processing
import sklearn.datasets
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from qiskit_machine_learning.datasets.dataset_helper import features_and_labels_transform

# for encoding functions from suzuki2020.pdf paper
import data_map_functions

Start by configuring joblib's caching and parallelism and the quantum backend. Parallelism and caching help save time and recover from power outages and shouldn't affect the final results. If the backend configuration is changed, the cache folder should be deleted so that the experiments run with the new backend and don't simply recall results from a previous backend.

In [2]:
# configure caching
cache_directory = "./joblib_cache"
memory = joblib.Memory(location=cache_directory, compress=False, mmap_mode=None)
# configure parallelism
worker_count = 6
parallel = joblib.Parallel(n_jobs=worker_count, backend="loky", batch_size=1, mmap_mode=None)
#configure backend
backend = Aer.get_backend("aer_simulator_statevector") # should configure this to mimic IBMQ backend
#backend.set_options(device='GPU')                      # enable GPU acceleration (comment this line to disable)

The functions in this cell are responsible for loading, preparing, and splitting data sets for input to the QSVM classifier.

In [3]:
def load_raw_datasets(seed=22):
    """This loads the sci-kit learn data set objects without performing any further
    processing on them. It also returns a readable name for each one."""
    return [sklearn.datasets.make_blobs(n_samples=150, n_features=4, random_state=seed), # create artificial blobs data set
            sklearn.datasets.load_breast_cancer(),
            sklearn.datasets.load_digits(),
            sklearn.datasets.load_iris(),
            sklearn.datasets.load_wine()]

def extract_binary_classes(feature_array, label_array):
    """Takes a numpy array of feature vectors and a numpy array of labels
    and returns transformed numpy arrays with the number of classes reduced
    to 2. Picks two random classes."""
    # get 2 random classes based on the seed, and output what they are
    all_classes = list(set(label_array))
    classes = np.random.choice(all_classes, size=2, replace=False) # must have replace=False to guarantee different classes
    print(f"Choosing classes {classes}.")
    class_map = {classes[0]:0, classes[1]:1} # convert labels to 0 and 1 (needed for training step)
    # construct a feature and label description with information from only the first 2 classes
    features = []
    labels = []
    for (feature, label) in zip(feature_array, label_array):
        if label in classes:
            features.append(feature)
            labels.append(label)
    return (np.array(features), np.array(labels))

def process_dataset(feature_vectors, labels, qubit_count, binary_classification=True):
    """Performs scaling and dimensionality reduction on all feature vectors of a data set,
    then returns the processed vectors. It will also extract 2 classes from the data set
    if binary_classification is True, rather than leaving the data set as a multi-class data set.
    Some of this code is modified from the qiskit_machine_learning data set loading
    source code (check qiskit_machine_learning.datasets.digits source code for exact location
    of what was modified from)."""
    # maybe extract classes for binary classification
    if binary_classification:
        feature_vectors, labels = extract_binary_classes(feature_vectors, labels)

    # Now we standardize for gaussian around 0 with unit variance
    scaler = StandardScaler()
    scaler.fit(feature_vectors)
    feature_vectors = scaler.transform(feature_vectors)

    # Now reduce number of features to number of qubits
    pca = PCA(n_components=qubit_count)
    pca.fit(feature_vectors)
    feature_vectors = pca.transform(feature_vectors)

    # Scale to the range (-1,+1)
    minmax_scaler = MinMaxScaler((-1, 1)).fit(feature_vectors)
    feature_vectors = minmax_scaler.transform(feature_vectors)

    # perform some other transformation on the feature and label vectors
    # as was done in the qiskit_machine_learning source code
    dataset_dict = {label:np.array([feature_vector for feature_vector, feature_vector_label in zip(feature_vectors, labels)
                                    if feature_vector_label == label])
                    for label in list(set(labels))}
    feature_vectors, labels = features_and_labels_transform(dataset_dict, labels, one_hot=False)

    return feature_vectors, labels

def cross_fold_sets(data, labels, k=5, seed=22):
    "Given a data set's feature and label arrays, yield training and testing feature and label arrays for k-fold validation. If the same seed is used then the same subsets should be returned across different calls."
    kf = KFold(n_splits=k, shuffle=True, random_state=seed)
    # for each of the k train-test splits:
    for train_indices, test_indices in kf.split(data, labels):
        # helper function
        extract_elements = lambda array, indices: np.array([array[i] for i in indices])
        # get training and testing feature vectors
        train_features = extract_elements(data, train_indices)
        test_features = extract_elements(data, test_indices)
        # get training and testing labels
        train_labels = extract_elements(labels, train_indices)
        test_labels = extract_elements(labels, test_indices)
        # return current split values
        yield (train_features, train_labels, test_features, test_labels)

This cell defines a function that can be given some parameters determining a classifier, like the feature map to use, the data to train on, and the backend to run the training on.

In [4]:
# MAYBE DO: make batch size a parameter
def make_classifier(feature_map_instance, training_features, training_labels):
    """Given a feature map instance, training features and labels,
    creates, trains, and returns a QSVM classifier."""
    # Create a quantum kernel from the feature map and
    # backend to give to the QSVC class.
    quantum_kernel = QuantumKernel(feature_map=feature_map_instance, quantum_instance=backend) # the batch_size argument is left as default since it just controls the number of circuits computed at once (irrelevant to results)
    # Create a QSVC instance
    qsvc = QSVC(quantum_kernel=quantum_kernel) # TODO: also set sklearn parameters here (look up the relevant ones)
    # Perform training
    qsvc.fit(training_features, training_labels)
    # return classifier instance
    return qsvc

This cell imports the five data mapping functions to be tested.

In [5]:
data_encoding_functions = [data_map_functions.data_map_one,
                           data_map_functions.data_map_two,
                           data_map_functions.data_map_three,
                           data_map_functions.data_map_four,
                           data_map_functions.data_map_five]
data_encoding_function_names = ["(8)", "(9)", "(10)", "(11)", "(12)"] # these names are the numberings used in the paper

This cell is similar to the above cell in that it in effect takes a specification for a classifier, but the function instead returns the generalisation metrics of the classifier that is described.

In [6]:
# TODO: finalize what generalisation metrics should be used and calculate them (maybe also look at margin size)
# MAYBE DO: put feature count in the argument list to make it and independent variable
# of the experiment rather than a constant.

# This class list is defined so that the feature map class argument to process_combination can be a number
# instead of a class instance. This is done to simplify caching.
feature_map_class_list = [qiskit.circuit.library.PauliFeatureMap,
                          qiskit.circuit.library.ZFeatureMap,
                          qiskit.circuit.library.ZZFeatureMap]
# This function is where most of the execution time
# lies since the model is also trained during its run time, so there
# is a large benefit to caching it to avoid recomputation after
# power failures. It is not a pure function however as it depends indirectly on the global
# values of feature_map_class_list and the backend used to compute the results, so
# the cache should be cleared if the backend or feature map list is changed.
# NOTE: caching doesn't work properly within a python notebook because of a joblib bug,
# so the notebook should be exported to a .py file then run if the ability to resume from
# interruptions is important.
# Define the uncached version of the function
def process_combination_uncached(feature_map_class_number, data_split_tuple, data_map_function, repetitions, qubit_count):
    """Takes a feature map class number, dataset loading function, and a backend, and
    returns the generalisation metrics of the combination of arguments."""
    # Create the feature map instance.
    feature_map_class = feature_map_class_list[feature_map_class_number]
    feature_map_instance = feature_map_class(feature_dimension=qubit_count, reps=repetitions, data_map_func=data_map_function)
    # unpack the data split for binary classification
    train_features, train_labels, test_features, test_labels = data_split_tuple

    # create the classifier
    qsvc = make_classifier(feature_map_instance, train_features, train_labels)

    # get the classification accuracy on training and testing data as generalisation metrics
    train_accuracy = qsvc.score(train_features, train_labels)
    test_accuracy = qsvc.score(test_features, test_labels)
    # return the generalisation metrics and the trained model
    return train_accuracy, test_accuracy, qsvc

# Define the main version of the function (process_combination), with cache use determined by a boolean variable.
# When caching is enabled, it saves results while processing for easy resuming from interruptions like power
# outages. Note that due to a joblib bug the code must be exported from the notebook to a .py file for caching to
# work. It might be fixed in a newer joblib version? (see this link for the bug:
# https://bleepcoder.com/joblib/608955182/using-memory-and-parallel-with-cached-function-defined)
use_checkpointing = True
process_combination = memory.cache(process_combination_uncached) if use_checkpointing else process_combination_uncached

This cell defines a function that collects the generalisation information of all tested classifier configurations.

In [7]:
# NOTE: these need to be manually edited to match the data set names and feature map names
dataset_names = ["blobs", "cancer", "digits", "iris", "wine"] # these should correspond to datasets returned by load_raw_datasets()
feature_map_names = ["Pauli", "Z", "ZZ"]
# This function takes a long time to run, so it benefits from parallelism.
# It is easily parallelised by running each data set's combinations with
# different workers.
# MAYBE DO: separate out the classifier configurations / combinations generation into
# a different function (for clarity). It could be called "generate_all_combinations"
# or something similar.
def process_all_combinations(do_binary_classification=True):
    """Runs all experiments."""
    # set a fixed seed so that caching works
    seed = 22
    np.random.seed(seed)      

    # get the list of dataset instances
    datasets = load_raw_datasets(seed=seed) # assigning to dataset_names edits the global variable

    # get a list of feature maps
    feature_map_class_numbers = range(len(feature_map_class_list)) # feature_map_class_list is a global list defined in the previous cell

    # Number of qubits to use / number of features to reduce to.
    qubit_count = 4

    # Define choice of k for k-fold cross-validation. This
    # number determines how many equally sized disjoint
    # subsets to split the dataset into, after which each
    # is used as the testing set in turn with the remaining
    # subsets being used as the training set.
    cross_validation_splits = 5 # a value of 5 gives a 20:80 test:train split, and 5 total validation splits
    
    # Do some output to the user to give them a sense of how long running the experiments will take
    number_of_investigated_repetition_values = 4 # for trying depth = 2, 3, 4, and 5
    combination_count = (len(datasets) * cross_validation_splits * len(data_encoding_functions) * number_of_investigated_repetition_values # number of ZZ combinations
                         + len(datasets) * cross_validation_splits * len(data_encoding_functions)  # number of Pauli combinations
                         + len(datasets) * cross_validation_splits)  # number of Z combinations
    print(f"Running with {len(datasets)} datasets, {len(feature_map_class_list)} feature maps, {number_of_investigated_repetition_values} different encoding repetitions for the ZZ feature map, {len(data_encoding_functions)} data map functions for the ZZ and Pauli feature maps, and {cross_validation_splits}-fold cross validation, requiring the training of {combination_count} classifiers in total.")

    # create a list of each possible combination / classifier
    # configuration so that they can be processed in parallel

    ## Process each combination.
    # First, make a list of all classifier configurations (combinations of
    # feature maps, training and testing splits, datasets, repetitions, etc.
    # - all identifying information for each classifier to create)
    print("Generating classifier configurations...")
    combinations = []
    for dataset, dataset_name in zip(datasets, dataset_names):
        # output dataset name
        print(f"{dataset_name} dataset:")
        # Perform dimensionality reduction and scaling on the features,
        # and transform the labels as done in the qiskit_machine_learning
        # data set loading source code. Also prepares the data for
        # binary rather than multi-class classification if enabled.
        if dataset_name == "blobs": # special case is for the blobs data set, which is not loaded as a dataset object
            blob_features, blob_labels = dataset # unpack pair
            features, labels = process_dataset(blob_features, blob_labels, binary_classification=do_binary_classification, qubit_count=qubit_count)
        else:
            features, labels = process_dataset(dataset.data, dataset.target, binary_classification=do_binary_classification, qubit_count=qubit_count)
        print(f"{len(features)} feature vectors.")
        # For each k-fold split of the data into training and testing sets
        for (split_number, split_tuple) in enumerate(cross_fold_sets(features, labels, k=cross_validation_splits)):
            # For each feature map
            for feature_map_class_number, feature_map_name in zip(feature_map_class_numbers, feature_map_names):
                if feature_map_name == "Z":
                    # for Z map, just do 2 repetitions with default encoding function (data_map_one, or data_encoding_functions[0])
                    process_combination_args = (feature_map_class_number, split_tuple, data_encoding_functions[0], 2, qubit_count) # to be passed to process_combination
                    identifying_key = (dataset_name, split_number, feature_map_name, data_encoding_function_names[0], 2) # to identify the combination in the results dictionary
                    combinations.append((process_combination_args, identifying_key))
                elif feature_map_name == "ZZ":
                    # for ZZ map, try all encoding functions and repetitions
                    for (data_encoding_function, data_encoding_function_name) in zip(data_encoding_functions, data_encoding_function_names):
                        for reps in range(2, 2+number_of_investigated_repetition_values):
                            process_combination_args = (feature_map_class_number, split_tuple, data_encoding_function, reps, qubit_count) # to be passed to process_combination
                            identifying_key = (dataset_name, split_number, feature_map_name, data_encoding_function_name, reps) # to identify the combination in the results dictionary
                            combinations.append((process_combination_args, identifying_key))
                elif feature_map_name == "Pauli":
                    # for Pauli map, try all encoding functions and 2 repetitions
                    for (data_encoding_function, data_encoding_function_name) in zip(data_encoding_functions, data_encoding_function_names):
                        process_combination_args = (feature_map_class_number, split_tuple, data_encoding_function, 2, qubit_count) # to be passed to process_combination
                        identifying_key = (dataset_name, split_number, feature_map_name, data_encoding_function_name, 2) # to identify the combination in the results dictionary
                        combinations.append((process_combination_args, identifying_key))
    # define a function to process the combinations information for a single classifier
    def results_from_combination(combination):
        # unpack the combination
        (pc_args, identifying_key) = combination
        # run experiment for this combination
        results = process_combination(*pc_args)
        # return results using human-readable names and values (other than the classifier instance, which is not human readable)
        return (identifying_key, results)

    # process each combination in parallel
    print("Creating, training, and testing classifiers...")
    key_value_result_pairs = parallel(joblib.delayed(results_from_combination)(combination) for combination in combinations)
    # create the results dictionary
    results_table = {}
    for (k, v) in key_value_result_pairs:
        results_table[k] = v
    return results_table

This cell defines a function that performs the experiments and reports their results.

In [8]:
models = []
def main ():
    results = process_all_combinations()
    for combination in results:
        train, test, model = results[combination]
        models.append((combination, model)) # store the model and the configuration used to create it
        print(f"Combination {combination} has train/test accuracies {train}/{test}.")
    joblib.dump(models, "trained_models.cache") # save all models to a file so they can be inspected later if desired
    return results

This cell can be evaluated to actually perform the experiments. It can take a few hours to run so loading pre-computed results is preferred.

In [ ]:
#results = main()                # uncomment this when you want to run it (to prevent running accidentally)

The next 3 cells should be run selectively to save and load pre-computed results. This caching is secondary to the memoization caching on individual functions, it just saves and loads the results variable only.

In [10]:
def save_results():
    """Saves results to a file if there are any, overwriting previous results."""
    if results != None:
        joblib.dump(results, "results.cache")
def load_results():
    """Loads results from a file."""
    global results
    results = joblib.load("results.cache")

In [11]:
#save_results() # uncomment this when you want to run it (to prevent accidentally running it)

In [14]:
#load_results() # uncomment this when you want to run it (to prevent accidentally running it)

These 2 cells read the results variable and combine cross-validation runs to get statistics.

In [12]:
def combine_cross_validations():
    def extract_cross_validations(combination):
        """Returns a list of training and a list of testing accuracies for the given
        combination, using the values in the results from the different cross-validation
        runs."""
        dataset_name, feature_map_name, data_encoding_function_name, repetitions = combination
        train_list = []
        test_list = []
        for key in results:
            (d_name, run, f_name, ef_name, rep) = key
            if d_name == dataset_name and f_name == feature_map_name and ef_name == data_encoding_function_name and rep == repetitions:
                train_accuracy, test_accuracy, model = results[key]
                train_list.append(train_accuracy)
                test_list.append(test_accuracy)
        return train_list, test_list
    # extract combinations to get statistics about from the results variable
    combinations = set()
    for key in results:
        (d_name, run, f_name, ef_name, rep) = key
        combinations.add((d_name, f_name, ef_name, rep))
    stats = {}
    for c in combinations:
        train_accuracies, test_accuracies = extract_cross_validations(c)
        stats[c] = ((np.mean(train_accuracies), np.std(train_accuracies), np.var(train_accuracies)),
                    (np.mean(test_accuracies), np.std(test_accuracies), np.var(test_accuracies)))
    return stats

This should only run after computing or loading a value for the results variable.

In [16]:
stats = combine_cross_validations()
number_of_investigated_repetition_values = 4
for dataset in dataset_names:
    for feature_map in feature_map_names:
        for repetition in range(2, (2+number_of_investigated_repetition_values) if feature_map == "ZZ" else 3):
            for data_encoding_function in (data_encoding_function_names[:1] if feature_map == "Z" else data_encoding_function_names): # restrict to first data encoding function if using the Z map with no entanglement, otherwise use all of them
                key = (dataset, feature_map, data_encoding_function, repetition)
                value = stats[key]
                print(f"Stats for {dataset} dataset, {feature_map} feature map, encoding function {data_encoding_function}, and {repetition} repetitions:")
                print("Mean classifier accuracies:")
                print(f"{value[0][0]} | {value[1][0]}")
                print("Standard deviations:")
                print(f"{value[0][1]} | {value[1][1]}")
                print("Variances:")
                print(f"{value[0][2]} | {value[1][2]}")
                print()

Stats for blobs dataset, Pauli feature map, encoding function (8), and 2 repetitions:
Mean classifier accuracies:
0.9200000000000002 | 0.6399999999999999
Standard deviations:
0.012747548783981964 | 0.020000000000000018
Variances:
0.00016250000000000005 | 0.0004000000000000007

Stats for blobs dataset, Pauli feature map, encoding function (9), and 2 repetitions:
Mean classifier accuracies:
0.99 | 0.7
Standard deviations:
0.009354143466934856 | 0.10488088481701516
Variances:
8.750000000000005e-05 | 0.011000000000000001

Stats for blobs dataset, Pauli feature map, encoding function (10), and 2 repetitions:
Mean classifier accuracies:
1.0 | 1.0
Standard deviations:
0.0 | 0.0
Variances:
0.0 | 0.0

Stats for blobs dataset, Pauli feature map, encoding function (11), and 2 repetitions:
Mean classifier accuracies:
0.9400000000000001 | 0.6900000000000001
Standard deviations:
0.021505813167606556 | 0.10677078252031309
Variances:
0.00046249999999999953 | 0.011399999999999997

Stats for blobs datas